In [ ]:
!pip install -U sentence-transformers
!pip install tensorflow_decision_forests
from sentence_transformers import SentenceTransformer
from google.colab import drive
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from nltk.corpus import stopwords
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, r2_score
import nltk
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4fc1d46e4b45a6ef28e3290efcb8fec66600ab89ce8fbce1d11ee5d50ca5ed29
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 37.3 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
drive.mount('/content/drive')
df_train_prompt = pd.read_csv('/content/drive/MyDrive/UOK/Level4/Research/Dataset/prompts_train.csv')
df_train_summaries = pd.read_csv('/content/drive/MyDrive/UOK/Level4/Research/Dataset/summaries_train.csv')
df_train = df_train_summaries.merge(df_train_prompt, on='prompt_id')

Mounted at /content/drive


In [ ]:
text_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Count the stop words in the text.
def count_stopwords(text: str) -> int:
    stopword_list = set(stopwords.words('english'))
    words = text.split()
    stopwords_count = sum(1 for word in words if word.lower() in stopword_list)
    return stopwords_count

# Count the punctuations in the text.
# punctuation_set -> !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
def count_punctuation(text: str) -> int:
    punctuation_set = set(string.punctuation)
    punctuation_count = sum(1 for char in text if char in punctuation_set)
    return punctuation_count

# Count the digits in the text.
def count_numbers(text: str) -> int:
    numbers = re.findall(r'\d+', text)
    numbers_count = len(numbers)
    return numbers_count

# This function applies all the above preprocessing functions on a text feature.
def feature_engineer(dataframe: pd.DataFrame, feature: str = 'text') -> pd.DataFrame:
    dataframe[f'{feature}_word_cnt'] = dataframe[feature].apply(lambda x: len(x.split(' ')))
    dataframe[f'{feature}_length'] = dataframe[feature].apply(lambda x: len(x))
    dataframe[f'{feature}_stopword_cnt'] = dataframe[feature].apply(lambda x: count_stopwords(x))
    dataframe[f'{feature}_punct_cnt'] = dataframe[feature].apply(lambda x: count_punctuation(x))
    dataframe[f'{feature}_number_cnt'] = dataframe[feature].apply(lambda x: count_numbers(x))
    return dataframe

preprocessed_df = feature_engineer(df_train)

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,text_word_cnt,text_length,text_stopword_cnt,text_punct_cnt,text_number_cnt
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,61,346,25,3,0
1,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,206,1225,92,30,3
2,0095993991fe,814d6b,The third wave only started as an experiment w...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,60,345,27,9,0
3,00c20c6ddd23,814d6b,The experimen was orginally about how even whe...,0.567975,0.969062,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,76,451,35,10,0
4,00d40ad10dc9,814d6b,The third wave developed so quickly due to the...,-0.910596,-0.081769,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,27,145,12,2,0


In [ ]:
df_train['cosine_similarity'] = np.nan
df_train['prompt_text_embeddings'] = np.empty((len(df_train),), dtype=object)
df_train['summary_text_embeddings'] = np.empty((len(df_train),), dtype=object)

for index, row in df_train.iterrows():

    prompt_text_embeddings = text_embedding_model.encode(row['prompt_text'])
    summary_text_embeddings = text_embedding_model.encode(row['text'])

    df_train.at[index, 'prompt_text_embeddings'] = prompt_text_embeddings
    df_train.at[index, 'summary_text_embeddings'] = summary_text_embeddings

    prompt_text_embeddings = prompt_text_embeddings.reshape(1, -1)
    summary_text_embeddings = summary_text_embeddings.reshape(1, -1)

    cosine_similarity_result = cosine_similarity(prompt_text_embeddings,summary_text_embeddings)

    df_train.at[index, 'cosine_similarity'] = cosine_similarity_result

    # if you need to track the progress, uncomment the following two lines
    # if (index % 1000 == 0):
    #   print(index)

df_train.to_pickle('your_dataframe.pkl')

0
1000
2000
3000
4000
5000
6000
7000


In [ ]:
df_train.head()

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,text_word_cnt,text_length,text_stopword_cnt,text_punct_cnt,text_number_cnt,cosine_similarity,prompt_text_embeddings,summary_text_embeddings
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,61,346,25,3,0,0.759260,"[-0.009068025, -0.043550443, -0.07066461, -0.0...","[-0.032301795, -0.061624367, -0.03943165, -0.0..."
1,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,206,1225,92,30,3,0.740786,"[-0.009068025, -0.043550443, -0.07066461, -0.0...","[-0.04587329, -0.07904928, -0.084160596, -0.02..."
2,0095993991fe,814d6b,The third wave only started as an experiment w...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,60,345,27,9,0,0.714380,"[-0.009068025, -0.043550443, -0.07066461, -0.0...","[-0.032329872, -0.06606195, -0.004307581, -0.0..."
3,00c20c6ddd23,814d6b,The experimen was orginally about how even whe...,0.567975,0.969062,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,76,451,35,10,0,0.563634,"[-0.009068025, -0.043550443, -0.07066461, -0.0...","[-0.08156457, 0.0050848215, -0.023151673, 0.06..."
4,00d40ad10dc9,814d6b,The third wave developed so quickly due to the...,-0.910596,-0.081769,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,27,145,12,2,0,0.670310,"[-0.009068025, -0.043550443, -0.07066461, -0.0...","[-0.018155737, -0.07647075, -0.01453981, -0.00..."


In [ ]:
def split_dataset(dataset, test_ratio=0.2):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(df_train)
train_ds_pd.shape, valid_ds_pd.shape

((5763, 16), (1402, 16))

In [ ]:
FEATURE_COLUMNS = [
    'cosine_similarity',
]

In [ ]:
# `content` label datatset columns
FEATURE_CONTENT = FEATURE_COLUMNS

# `wording` label datatset columns
FEATURE_WORDING = FEATURE_COLUMNS

# Convert dataframes to corresponding datasets
content_train_ds_x = np.array(train_ds_pd[FEATURE_CONTENT])
content_train_ds_y = np.array(train_ds_pd['content'])

content_valid_ds_x =  np.array(valid_ds_pd[FEATURE_CONTENT])
content_valid_ds_y = np.array(valid_ds_pd['content'])




wording_train_ds_x = np.array(train_ds_pd[FEATURE_WORDING])
wording_train_ds_y = np.array(train_ds_pd['wording'])

wording_valid_ds_x = np.array(valid_ds_pd[FEATURE_WORDING])
wording_valid_ds_y = np.array(valid_ds_pd['wording'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_content = RandomForestRegressor()
model_wording = RandomForestRegressor()

In [ ]:
# Training for label content
model_content.fit(content_train_ds_x, content_train_ds_y)

# Training for label wording
model_wording.fit(wording_train_ds_x, wording_train_ds_y)

RandomForestRegressor()

In [ ]:
content_y_pred = model_content.predict(content_valid_ds_x.reshape(-1,1))

wording_y_pred = model_wording.predict(wording_valid_ds_x.reshape(-1,1))

In [ ]:
content_mse = mean_squared_error(content_valid_ds_y, content_y_pred)
content_r2 = r2_score(content_valid_ds_y, content_y_pred)

print(f'Mean Squared Error - Content: {content_mse}')
print(f'R-squared - Content: {content_r2}')


wording_mse = mean_squared_error(wording_valid_ds_y, wording_y_pred)
wording_r2 = r2_score(wording_valid_ds_y, wording_y_pred)

print(f'Mean Squared Error - Wording: {wording_mse}')
print(f'R-squared - Wording: {wording_r2}')

Mean Squared Error - Content: 1.1671890165017125
R-squared - Content: -0.03546001070138827
Mean Squared Error - Wording: 1.4220513996066562
R-squared - Wording: -0.2915877849453954


In [ ]:
print(content_y_pred[:10])
print(content_valid_ds_y[:10])

[ 0.16050409  0.26796304 -0.04294803  0.02714189  1.03542346 -1.16141444
 -1.17976647  1.61531134 -0.45458196  0.26793212]
[ 3.27289415  0.20568251 -0.32815099  1.40779064 -0.39331017 -0.4194991
 -0.40380165  1.1082943  -1.18487082  2.83495109]
